In [1]:
import argparse, os, sys, datetime, glob, importlib, csv,logging
import numpy as np
import time
import torch
import torchvision
import torch.nn as nn
from torch import optim
import pytorch_lightning as pl
from tqdm import tqdm
import anndata
# from packaging import version
from omegaconf import OmegaConf
# from torch.utils.data import random_split, DataLoader, Dataset, Subset
# from functools import partial
# from PIL import Image

# from pytorch_lightning import seed_everything
# from pytorch_lightning.trainer import Trainer
# from pytorch_lightning.callbacks import ModelCheckpoint, Callback, LearningRateMonitor
# from pytorch_lightning.utilities.distributed import rank_zero_only
# from pytorch_lightning.utilities import rank_zero_info

# from stable_diffusion_main.ldm.data.base import Txt2ImgIterableBaseDataset
from diffusion.util import instantiate_from_config

from sklearn.model_selection import KFold
from pathlib import Path
from torch import optim
from torch.utils.data import SubsetRandomSampler,DataLoader
from torchsummary import summary
import onnx
import onnx.utils
import onnx.version_converter

/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import scanpy as sc
import scipy.sparse as sp
from preprocess import construct_interaction,construct_interaction_KNN,preprocess,get_feature,preprocess_adj_sparse,preprocess_adj,mask_nodes_edges
from gae.model import GCNModelVAE_FC
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from diffusion.ddpm import Diffusion
from diffusion.data import myDataset
from diffusion.ddim import DDIMSampler

In [4]:
device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")
# logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "-b",
    "--base",
    nargs="*",
    metavar="base_config.yaml",
    help="paths to base configs. Loaded from left-to-right. "
         "Parameters can be overwritten or added with command-line options of the form `--key value`.",
    default=list(["./configs/v1-inference.yaml",]),
)
datatype='10x'

In [6]:
opt, unknown = parser.parse_known_args()
configs = [OmegaConf.load(cfg) for cfg in opt.base]
cli = OmegaConf.from_dotlist(unknown)
config = OmegaConf.merge(*configs, cli)
# config.unet_config['params']['use_spatial_transformer']=False##########改
config.unet_config

{'target': 'diffusion.openaimodel.UNetModel', 'params': {'image_size': 32, 'in_channels': 1, 'out_channels': 1, 'model_channels': 256, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 128, 'use_checkpoint': True, 'legacy': False}}

In [7]:
adata = sc.read_visium('../data/ST-colon3',
           count_file='filtered_feature_bc_matrix.h5'
                      )
adata.var_names_make_unique()

/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [7]:
# data_path = '../data/Human_ColorectalCancer'
# count_file = 'Targeted_Visium_Human_ColorectalCancer_GeneSignature_filtered_feature_bc_matrix.h5'
# source_image_path = 'Targeted_Visium_Human_ColorectalCancer_GeneSignature_image.tif'

data_path = '../data/human_breast_cancer'
count_file = 'V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5'
source_image_path = 'V1_Breast_Cancer_Block_A_Section_1_image.tif'
#human_ovarian_cancer_target
# data_path = '../data/human_ovarian_cancer_target'
# count_file = 'Targeted_Visium_Human_OvarianCancer_Pan_Cancer_filtered_feature_bc_matrix.h5'
# source_image_path = 'Targeted_Visium_Human_OvarianCancer_Pan_Cancer_image.tif'
adata = sc.read_visium(data_path,
           count_file= count_file
           ,source_image_path = source_image_path
          )

/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [7]:
matrix_file = '../data/HT231P1/filtered_feature_bc_matrix/'
# Read the files and create an AnnData object
adata = sc.read_10x_mtx(matrix_file, var_names='gene_symbols', cache=True)
adata.var_names_make_unique()

# Replace with the actual path to your spatial coordinates file
spatial_coordinates_file = '../data/HT231P1/spatial/tissue_positions_list.csv'

# # Read spatial coordinates
spatial_coordinates = pd.read_csv(spatial_coordinates_file,header=None, index_col=0)
column_names = ['in_tissue','array_col','array_row','y','x']  # Replace with your actual column names
spatial_coordinates.columns = column_names
spatial_coordinates = spatial_coordinates.reindex(adata.obs_names)
# Add spatial coordinates to adata
adata.obs['in_tissue'] = spatial_coordinates['in_tissue']
adata.obs['array_col'] = spatial_coordinates['array_col']
adata.obs['array_row'] = spatial_coordinates['array_row']
adata.obs['x'] = spatial_coordinates['x']
adata.obs['y'] = spatial_coordinates['y']
spatial_array = spatial_coordinates[['y', 'x']].values

# Assign the spatial array to adata.obsm['spatial']
adata.obsm['spatial'] = spatial_array 
adata = adata[adata.obs['in_tissue']==1,:]

/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [8]:
# data_path = '../data/mouse_B06'
# count_file = 'B06_E1.h5ad'
# source_image_path = 'B06_E1__HE.png'
# adata = sc.read_h5ad(os.path.join(data_path,count_file))
# adata.obsm['spatial']=adata.obs[['img_coord_X','img_coord_Y']].values.astype(np.int)

In [8]:
#preprocess data
if 'highly_variable' not in adata.var.keys():
    preprocess(adata)

if 'adj' not in adata.obsm.keys():
    if datatype in ['Stereo', 'Slide']:
        construct_interaction_KNN(adata)
    else:    
      # construct_interaction(adata)
        construct_interaction_KNN(adata)

if 'feat' not in adata.obsm.keys():
    get_feature(adata)

/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:61: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Graph constructed!


In [9]:
#graph feature
features = torch.tensor(adata.obsm['feat'].copy()).to(device)
adj = adata.obsm['adj']
graph_neigh = torch.tensor(adata.obsm['graph_neigh'].copy() + np.eye(adj.shape[0])).to(device)

#get adj
if datatype in ['Stereo', 'Slide']:
    #using sparse
    print('Building sparse matrix ...')
    adj_norm = preprocess_adj_sparse(adj).to(device)
    pos_weight = torch.tensor(float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum())
    norm = torch.tensor(adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2))
    adj = torch.tensor(adj+ sp.eye(adj.shape[0])).to(device)
else: 
    # standard version
    # adj_norm = preprocess_adj(adj)
    # adj_norm = torch.FloatTensor(adj_norm).to(device)
    adj_norm = preprocess_adj_sparse(adj).to(device)
    pos_weight = torch.tensor(float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum())
    norm = torch.tensor(adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2))
    adj = torch.tensor(adj+ sp.eye(adj.shape[0])).to(device)

In [10]:
num_features = features.shape[1]
hidden1=512 #Number of units in hidden layer 1
hidden2=128 #Number of units in hidden layer 2
fc_dim1=512 #Number of units in the fully connected layer of the decoder
dropout=0.01 #neural network dropout term
firststagemodel = GCNModelVAE_FC(num_features, hidden1,hidden2,fc_dim1, dropout).to(device)
# firststagemodel.load_state_dict(torch.load('./models/HT231P1/get_gae_feature/HT231P1_gae.ckpt'))
# firststagemodel.load_state_dict(torch.load('./models/HT231P1/get_gae_feature/HT231P1_gae.ckpt'))
firststagemodel.load_state_dict(torch.load('./models/human_breast_cancer/get_gae_feature/breast_gae_010.ckpt'))
# firststagemodel.load_state_dict(torch.load('./models/ST-colon3/get_gae_feature/gae.pt'))

firststagemodel.eval()
adj_recon,mu,logvar,z, features_recon = firststagemodel(features, adj_norm)

In [11]:
image_save_path = Path('./models/human_breast_cancer/get_img_feature') / ("image_feature.npy")
# image_save_path = Path('./models/HT231P1/get_img_feature') / ("image_feature.npy")
# image_save_path = Path('../data/human_ovarian_cancer_target') / ("img_feature_128.npy")
# image_save_path = Path('./models/ST-colon3/get_img_feature') / ("image_feature.npy")
image_feature = torch.tensor(np.load(image_save_path))

In [12]:
z = z.detach().cpu()

In [13]:
myData = myDataset(z,image_feature)
diffusion = Diffusion(image_size=128)
learning_rate = 1e-4
learning_rate = 1e-4
batch_size = 64
epochs = 300
num_timesteps =1000
ddim_steps = 50
strength = 0.5
scale = 5.0

/home/sqjiao/IDDPM/stable_diffusion/diffusion/data.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(condition)


In [14]:
mes_mtx = np.zeros((len(myData),z.shape[1]))
com_mtx = np.zeros((len(myData),z.shape[1]))

In [ ]:
k = 5
splits=KFold(n_splits=k,shuffle=True)
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(myData)))):
    
    print('Fold {}'.format(fold + 1))
    model = instantiate_from_config(config.unet_config).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    mse = nn.MSELoss()
    train_sampler = SubsetRandomSampler(train_idx)##################################
    test_sampler = val_idx
    train_loader= DataLoader(myData, batch_size=batch_size, shuffle=False,sampler = train_sampler)
    test_loader = DataLoader(myData, batch_size=batch_size, shuffle=False,sampler = test_sampler)
    l = len(train_loader)
    for epoch in range(epochs):
        logging.info(f"Starting epoch {epoch+1}:")
        pbar = tqdm(train_loader)
        #########################################
        for i,(model_input,cond_input) in enumerate(train_loader): 
            # print(model_input.shape)
            t = torch.randint(0, num_timesteps, (model_input.shape[0],)).long()
            noise = torch.randn_like(model_input)
            x_t = diffusion.q_sample(model_input, t,noise)
            
            noise = noise.to(device)
            x_t = x_t.to(device)
            t = t.to(device)
            
            cond_input = cond_input.to(device)
            predicted_noise = model(x_t, t, cond_input)#######改
            # predicted_noise = model(x_t, t)
            loss = mse(noise, predicted_noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(MSE=loss.item())
            # logger.add_scalar("MSE", loss.item(), global_step=epoch * l+i )

    model.eval()
    sampler = DDIMSampler(model,diffusion,device=device,num_timesteps=num_timesteps)
    sampler.make_schedule(ddim_num_steps=ddim_steps, verbose=False)
    t_enc = int(strength * ddim_steps)
    with torch.no_grad():
        for i,(model_input,cond_input) in enumerate(test_loader):
            model_input = model_input.to(device)
            cond_input = cond_input.to(device)
            z_enc = sampler.stochastic_encode(model_input, torch.tensor([t_enc]*model_input.shape[0]).to(device))
            samples = sampler.decode(z_enc, cond_input, t_enc)####################改
            # samples = sampler.decode(z_enc ,_ , t_enc)

            mes_res = model_input.reshape((model_input.shape[0],model_input.shape[2])).to('cpu').numpy()
            com_res = samples.reshape((samples.shape[0],samples.shape[2])).to('cpu').numpy()
            mes_mtx[test_sampler[0+batch_size*i:model_input.shape[0]+batch_size*i],:] = mes_res  
            com_mtx[test_sampler[0+batch_size*i:model_input.shape[0]+batch_size*i],:] = com_res
        # mes_mtx.write(os.path.join("Spatial/osmFISH/results", "measure", f"Fold{fold+1}_test_nofea_addpos_per50_{epoch}.h5ad"))
        # com_mtx.write(os.path.join("Spatial/osmFISH/results", "predict", f"Fold{fold+1}_test_nofea_addpos_per50_{epoch}.h5ad"))
    model.train()

Fold 1


  0%|          | 0/48 [00:26<?, ?it/s, MSE=0.163]

  0%|          | 0/48 [00:23<?, ?it/s, MSE=0.0595]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0689]

  0%|          | 0/48 [00:23<?, ?it/s, MSE=0.0413]

  0%|          | 0/48 [00:23<?, ?it/s, MSE=0.0457]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0374]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0444]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0401]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0295]

  0%|          | 0/48 [00:23<?, ?it/s, MSE=0.042]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0347]

  0%|          | 0/48 [00:21<?, ?it/s, MSE=0.0172]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0256]

  0%|          | 0/48 [00:23<?, ?it/s, MSE=0.0393]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.035]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0317]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0269]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0291]

  0%|          | 0/48 [00:22<?, ?it/s, MSE=0.0155]

  0%|          

In [ ]:
sam = mes_mtx

In [ ]:
adj_norm

In [ ]:
z

In [ ]:
com_fin = firststagemodel.decoder(torch.tensor(com_mtx).to(torch.float32).to(device), adj_norm)

In [ ]:
mes_fin = firststagemodel.decoder(torch.tensor(z).to(torch.float32).to(device), adj_norm)

In [ ]:
sam = mes_fin[3].detach().cpu().numpy()

In [ ]:
com = com_fin[3].detach().cpu().numpy()

In [ ]:
# sam = z
# com = com_mtx

In [ ]:
pic = ['Lamp5','Kcnip2','Rorb','Trf','Syt6','Acat2','Acot13','Septin4']
plt.style.use('dark_background')
for i in range(20):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(8,4))
    ax1.axis('off')
    cmap = sam[:,i]
    cmap[cmap > np.percentile(cmap,99)] = np.percentile(cmap,99)
    ax1.scatter(adata.obsm['spatial'][:,0],adata.obsm['spatial'][:,1],s=1,c=cmap)
    ax1.set_title('Measured ', fontsize = 12)
    ax1.set_ylabel(i)
    ax1.invert_yaxis()
    # ax1.invert_xaxis()
    ax2.axis('off')
    cmap = com[:,i]
    cmap[cmap > np.percentile(cmap,99)] = np.percentile(cmap,99)
    ax2.scatter(adata.obsm['spatial'][:,0],adata.obsm['spatial'][:,1],s=1,c=cmap)
    ax2.set_title('Predicted ', fontsize = 12)
    ax2.invert_yaxis()
    # ax2.invert_xaxis()

In [ ]:
result = anndata.AnnData(com)

In [ ]:
result.write(os.path.join("./outputs/human_breast_cancer/", f"per40_epoch250_imgfeature_drop010.h5ad"))